<a href="https://colab.research.google.com/github/zebedy-magic/BottleImageRecognition/blob/master/Bottle_recognition_part3_identify_bottles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 3 - Identification time !!


Lets try and see if we can identify a bottle compared to anything else so this is a binary problem..is it a bottle ? Yes/No 

We will try an add some more features such as the HOG algorithm 

Do we need color ? 
Can we pass the image in as is as an array 
Do we need other features added ? 

Lets try simple an SVM with just with gray images passed in 





In [0]:
import pandas as pd 
from os.path import join
from PIL import Image
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# import train_test_split from sklearn's model selection module
from sklearn.model_selection import train_test_split

# import SVC from sklearn's svm module
from sklearn.svm import SVC

# import accuracy_score from sklearn's metrics module
from sklearn.metrics import roc_curve, auc, accuracy_score


#root directory
root='/content/drive/My Drive/'

image_path=join(root,'images','processed')


In [0]:
def get_image(filename,image_path):
  #get the image
  img=Image.open(join(image_path,filename))
  return np.array(img)

In [0]:
#lets get image and create some features (first lets just flatten and nothing more )

def create_features(img):
  #lets just flatten the image and thats it for now 
  feature=img.flatten()
  
  return feature


In [0]:
def create_matrix(labels):
  #create feature matrix using the creat features 
  lst_images=[]
  #loop through the images .get the features and append to a list
  for image in labels.index:
    img_arr=get_image(image,image_path)
    
    feature=create_features(img_arr)
    
    lst_images.append(feature)
  #use array to make it into one big matrix
  matrix=np.array(lst_images)
  return matrix

In [0]:
def get_ss_pca(matrix):
  
  #lets use the standard scaler to scale for algorithm
  ss=StandardScaler()
  ss_matrix=ss.fit_transform(matrix)

  #this reduces the number of features so less power needed to work on problem 
  pca=PCA(n_components=24)
  ss_pca=pca.fit_transform(ss_matrix)
  return ss_pca

  

In [0]:
#lets get in the labels as a DataFrame 
#

labels=pd.read_csv(join(root,'labels.csv'),index_col=0)
print(labels.head())
print(labels.info())



                                 bottle
image                                  
2020-04-25 12:01:24.739043.jpeg       1
2020-04-25 12:01:26.103276.jpeg       0
2020-04-25 12:01:27.977304.jpeg       0
2020-04-25 12:01:29.199803.jpeg       1
2020-04-25 12:01:30.760437.jpeg       0
<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 2020-04-25 12:01:24.739043.jpeg to 2020-04-26 10:38:31.630673.jpeg
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   bottle  24 non-null     int64
dtypes: int64(1)
memory usage: 384.0+ bytes
None


In [0]:
#create the matrix of features
matrix=create_matrix(labels)
print(matrix.shape)

#create the standardisatiom and PCA matrix of features
ss_matrix=get_ss_pca(matrix)
print(ss_matrix.shape)

(24, 40000)
(24, 24)


In [0]:
#lets do the train test split ... the matrix of images is in the same order as the labels DF
 

X_train, X_test, y_train, y_test = train_test_split(ss_matrix,
                                                    labels.bottle.values,
                                                    test_size=.3,
                                                    )

In [0]:
#Lets do the SVM 
#as this is binary going to use SVC (support vector classifier)
svc=SVC(kernel='linear',probability=True)
svc.fit(X_train,y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
#lets see the accuracy 
pred=svc.predict(X_test)

accuracy=accuracy_score(pred,y_test)

print('Accuracy is {0} '.format(accuracy))

#to do ROC 

[1 0 0 0 0 1 1 0] [1 0 0 0 0 1 1 0]
Accuracy is 1.0 


1st try : Wow 100% accuracy with just using a matrix of a grayscaled image .. maybe we are overfitting so need to try more images than 24 